In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
lstm= nn.LSTM(3,3) # input dim and output dim =3

inputs=[torch.randn(1,3) for _ in range(5)] # make a sequence of length 5
hidden=(torch.randn(1,1,3), torch.randn(1,1,3)) #initialize a hidden state

for i in inputs:
  out, hidden=lstm(i.view(1,1,-1), hidden)

inputs=torch.cat(inputs).view(len(inputs), 1, -1)
hidden=(torch.randn(1, 1, 3), torch.randn(1, 1, 3)) # clear out hidden states

out, hidden=(inputs, hidden)

print(f"Output: {out}")
print(f"Hidden: {hidden}")





Output: tensor([[[ 0.2840, -0.5457, -0.7092]],

        [[-0.0397,  0.6734,  0.2899]],

        [[-0.6523,  0.4644, -0.7886]],

        [[-0.1547, -0.6124, -1.1373]],

        [[-1.9719,  2.4837, -0.2139]]])
Hidden: (tensor([[[1.6444, 0.3410, 1.0874]]]), tensor([[[ 0.2515, -0.0496,  2.1962]]]))


In [0]:
def prepare_sequence(sequence,to_ix):
  idx=[to_ix[w] for w in sequence]
  return torch.tensor(idx,dtype=torch.long)


training_data=[("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
               ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
               ]

word_to_ix={}

for sent, tags in training_data:
  for word in sent:
    if word not in word_to_ix:
      word_to_ix[word]=len(word_to_ix)

print(word_to_ix)

tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [0]:
class LSTMTagger(nn.Module):
  
  def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
    super(LSTMTagger, self).__init__()


    self.hidden_size=hidden_dim
    self.word_embeddings=nn.Embedding(vocab_size, embedding_dim)
    self.lstm=nn.LSTM(embedding_dim, hidden_dim)

    self.hidden_to_tag= nn.Linear(hidden_dim, tagset_size)

  def forward(self, sentence):
    embeds=self.word_embeddings(sentence)
    lstm_out, _=self.lstm(embeds.view(len(sentence), 1, -1))

    tag_space=self.hidden_to_tag(lstm_out.view(len(sentence), -1))

    tag_score=F.softmax(tag_space, dim=1)

    return tag_score






In [0]:
model=LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))

loss_function=nn.NLLLoss()

optimizer=optim.SGD(model.parameters(), lr=0.1)

with torch.no_grad():
  inputs=prepare_sequence(training_data[0][0], word_to_ix)
  tag_score=model(inputs)
  print(tag_score)



for epoch in range(30):
  for sentence, tags in training_data:
    model.zero_grad() # clear gradients since pytorch accumulate gradient
    sentence_in = prepare_sequence(sentence, word_to_ix)
    targets= prepare_sequence(tags, tag_to_ix)

    tag_scores=model(sentence_in)

    loss=loss_function(tag_scores, targets)
    loss.backward()
    optimizer.step()
        
with torch.no_grad():
  inputs=prepare_sequence(training_data[0][0], word_to_ix)
  tag_scores=model(inputs)
  print(tag_scores)








tensor([[0.2444, 0.3690, 0.3866],
        [0.2258, 0.3422, 0.4320],
        [0.2208, 0.3468, 0.4324],
        [0.2387, 0.3381, 0.4232],
        [0.2225, 0.3438, 0.4336]])
tensor([[0.2408, 0.4800, 0.2792],
        [0.2213, 0.4485, 0.3303],
        [0.2140, 0.4557, 0.3303],
        [0.2404, 0.4315, 0.3281],
        [0.2176, 0.4568, 0.3256]])
